In [1]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(
    dbname="recruiting",
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port=6543  # local port you forwarded
)
query = """
WITH edu AS (
    SELECT
        username,
        COALESCE(
            string_agg(
                format(
                    'edu: id=%s, start_date=%s, end_date=%s, fieldOfStudy=%s, degree=%s, grade=%s, schoolName=%s, description=%s, activities=%s, schoolId=%s',
                    id, start_date, end_date, "fieldOfStudy", degree, grade, "schoolName", description, activities, "schoolId"
                ),
                ' | '
            ),
            ''
        ) AS edu_text
    FROM education_data
    GROUP BY username
),
pos AS (
    SELECT
        username,
        COALESCE(
            string_agg(
                format(
                    'pos: id=%s, companyId=%s, companyName=%s, companyUsername=%s, companyIndustry=%s, companyStaffCountRange=%s, title=%s, location=%s, description=%s, employmentType=%s, start_date=%s, end_date=%s',
                    id, "companyId", "companyName", "companyUsername", "companyIndustry", "companyStaffCountRange", title, location, description, "employmentType", start_date, end_date
                ),
                ' | '
            ),
            ''
        ) AS pos_text
    FROM position_data
    GROUP BY username
)
SELECT
    p.id,
    p."fullName",
    p.summary,
    p.skills,
    p.location,
    p.country,
    p.city,
    concat_ws(' | ', edu_text, pos_text) AS combined_text
FROM person_data p
LEFT JOIN edu ON p.username = edu.username
LEFT JOIN pos ON p.username = pos.username;
"""




df_raw = pd.read_sql_query(query, conn)
import json
# Create the 'text' column by concatenating all fields into a string
df = df_raw.copy()
df

/var/folders/0_/14m9hb7n2234bn6kbkgxxvbr0000gn/T/ipykernel_41284/832720302.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql_query(query, conn)


,id,fullName,summary,skills,location,country,city,combined_text
0,291,Ray Guerrero,Product and business developer specialized in ...,"Finanzas, Contabilidad, Análisis financiero, A...",Mexico,Mexico,,"edu: id=740, start_date=2017-08-01 00:00:00, e..."
1,64,Dexter Zhuang,"👋 Hi! I’m Dexter, founder of Portfolio Path an...","Communication, Analytical Skills, Critical Thi...","San Francisco, CA",United States,"San Francisco, California","edu: id=160, start_date=, end_date=, fieldOfSt..."
2,292,Debi Çakar,🎓 Bahçeşehir University graduate with a double...,"Technical Knowledge, Scrum and Agile developme...","Istanbul, Türkiye",Türkiye,Istanbul,"edu: id=742, start_date=2016-01-01 00:00:00, e..."
3,293,Sophy Su Yadana Zaw,I am a dynamic and results-oriented Product Ma...,"Project Management, Process Management, Agile ...",Bangkok,Thailand,"Bangkok, Bangkok City","edu: id=744, start_date=, end_date=, fieldOfSt..."
4,79,Büşra SEY,Experienced Product and Buying Manager with ov...,"Tedarikçi Yönetimi, Fiyatlandırma Stratejisi, ...","Istanbul, Türkiye",Türkiye,Istanbul,"edu: id=205, start_date=2008-01-01 00:00:00, e..."
...,...,...,...,...,...,...,...,...
361,285,Paweł Huryn,Follow me for actionable insights and resource...,"Product Strategy, Artificial Intelligence (AI)...",Warsaw,Poland,"Warsaw, Mazowieckie","edu: id=724, start_date=2011-01-01 00:00:00, e..."
362,286,Roberta R. Rosário Gimenez,I’m a product leader with 15+ years of experie...,"New Business Development, Business Case Develo...","São Paulo, Brazil",Brazil,São Paulo,"edu: id=725, start_date=2013-01-01 00:00:00, e..."
363,284,Monika Goszcz,I professionally make impactful changes with p...,"Planowanie budżetu, Inżynieria, Zarządzanie, P...",Poland,Poland,,"edu: id=721, start_date=2017-01-01 00:00:00, e..."
364,289,Arslan Kiran,Leading growth in Turkey at Moca Network at An...,"Growth Strategies, Web3, mobile gaming, Digita...",Istanbul,Türkiye,"Istanbul, Istanbul","edu: id=735, start_date=2007-01-01 00:00:00, e..."


In [2]:
# Assuming df_raw is the DataFrame with columns:
# ['id', 'fullName', 'summary', 'skills', 'location', 'country', 'city', 'combined_text']

# Create a new column "text" by concatenating selected columns (you may adjust the order and formatting as needed)
df["text"] = df.apply(lambda row: "\n".join([
    f"fullName: {row['fullName']}",
    f"summary: {row['summary']}",
    f"skills: {row['skills']}",
    f"location: {row['location']}",
    f"country: {row['country']}",
    f"city: {row['city']}",
    f"combined_text: {row['combined_text']}"
]), axis=1)

# Create new DataFrame with only 'id' and the newly created 'text' column
df = df[["id", "text"]]

# Optionally, to inspect the result without truncation, set Pandas display options:
import pandas as pd
pd.set_option('display.max_colwidth', None)
print(df.head())


    id  \
0  291   
1   64   
2  292   
3  293   
4   79   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [3]:
df.iloc[0]["text"]

'fullName: Ray Guerrero\nsummary: Product and business developer specialized in the Fintech industry. Skilled in implementing from zero to one debit and credit card products, e-wallet innovation, international digital remittances, and payment functionalities.\nskills: Finanzas, Contabilidad, Análisis financiero, Análisis, Precios de transferencia, SAP, Finanzas corporativas, Pronósticos\nlocation: Mexico\ncountry: Mexico\ncity: \ncombined_text: edu: id=740, start_date=2017-08-01 00:00:00, end_date=2019-07-01 00:00:00, fieldOfStudy=, degree=Executive MBA, grade=, schoolName=IPADE Business School, description=, activities=, schoolId=15106574 | edu: id=741, start_date=2007-01-01 00:00:00, end_date=2012-01-01 00:00:00, fieldOfStudy=, degree=Accounting & Finance, grade=, schoolName=Instituto Tecnológico Autónomo de México, description=Financial vision of Accounting, activities=, schoolId=15249288 | pos: id=1798, companyId=27227313, companyName=FinTech México , companyUsername=asociacion-fin

In [4]:
import pandas as pd
import requests
import dotenv

dotenv.load_dotenv()
# Example dataframe

vacancy_text = """
Название должности: Product Manager
Альтернативные тайтлы: Country Manager, Business Development Manager, Regional Manager
Опыт в годах: 3
Обязательные навыки: Product development, Advanced Spanish, Latam region, Native Russian
Дополнительно важно: важен русскоговорящий к-т
Полезные навыки: New markets, Sales management, market research, budgeting
Домены: must have  iGaming, Gambling
Локации поиска: Кипр, Латвия, Грузия, Мальта, Польша, Сербия, Мексика, Испания, Эстония, Россия, Болгария, Литва, Казахстан, Португалия, Таиланд, Бразилия, Аргентина, Чили, Турция
Гражданство: не Армении, не Грузии

"""

# Convert dataframe to list of dicts
candidates = df.to_dict(orient="records")

# Request payload
payload = {
    "vacancy_text": vacancy_text,
    "candidates": candidates
}

# Send POST request
response = requests.post("http://localhost:8000/match_candidates", json=payload)
print(response.text)


{"custom_id":{"0":"291","1":"64","2":"292","3":"293","4":"79","5":"241","6":"294","7":"10","8":"51","9":"6","10":"7","11":"52","12":"295","13":"5","14":"296","15":"22","16":"54","17":"55","18":"297","19":"8","20":"208","21":"1","22":"299","23":"298","24":"300","25":"2","26":"301","27":"302","28":"303","29":"304","30":"14","31":"214","32":"68","33":"82","34":"81","35":"210","36":"216","37":"84","38":"85","39":"217","40":"306","41":"307","42":"308","43":"211","44":"12","45":"92","46":"309","47":"25","48":"15","49":"60","50":"61","51":"207","52":"96","53":"310","54":"83","55":"17","56":"311","57":"95","58":"98","59":"312","60":"313","61":"100","62":"50","63":"314","64":"86","65":"101","66":"97","67":"31","68":"103","69":"18","70":"104","71":"105","72":"221","73":"106","74":"59","75":"107","76":"316","77":"108","78":"212","79":"317","80":"318","81":"223","82":"53","83":"222","84":"62","85":"322","86":"225","87":"115","88":"326","89":"327","90":"39","91":"58","92":"93","93":"328","94":"94",

In [5]:
import json
data = json.loads(response.text)

# The response contains two keys: "custom_id" and "content",
# each mapping indices to the appropriate values.
rows = []
# Loop over the sorted keys (as integers) to maintain order.
for idx in sorted(data["custom_id"], key=lambda x: int(x)):
    custom_id = data["custom_id"][idx]
    # The 'content' is itself a JSON string (structured output)
    content_str = data["content"][idx]
    try:
        # Convert the JSON string to an object (dictionary)
        content_obj = json.loads(content_str)
    except json.JSONDecodeError:
        content_obj = content_str
    rows.append({
        "custom_id": custom_id,
        "content": content_obj
    })

# Create DataFrame:
df_results = pd.DataFrame(rows)
df_results

,custom_id,content
0,291,"{'experience': {'feature': 'Experience', 'score': 5, 'explanation': 'The candidate has around 6 years of experience in product management and business development, which exceeds the required 3 years. However, this could indicate overqualification.'}, 'skills': {'feature': 'Mandatory skills', 'score': 0, 'explanation': 'The candidate does not meet the mandatory skills requirements of 'Product development', 'Advanced Spanish', or 'Native Russian'. This strongly impacts the evaluation.'}, 'domain': {'feature': 'Domain Expertise in iGaming, Gambling', 'score': 0, 'explanation': 'The candidate has no experience or expertise in the iGaming or Gambling domains, which are critical for this role.'}, 'location': {'feature': 'Location', 'score': 10, 'explanation': 'The candidate is located in Mexico, which is one of the preferred locations for the job.'}, 'overall_summary': {'score': 5, 'summary': 'Overall, the candidate has significant experience and is located in a preferred region, but lacks the mandatory skills and domain expertise required for the role.'}}"
1,64,"{'experience': {'feature': 'Years of Experience', 'score': 5, 'explanation': 'The candidate has 12 years of experience which is significantly above the required 3 years. While this is generally positive, the overqualification could imply a mismatch with expectations.'}, 'skills': {'feature': 'Mandatory Skills', 'score': 0, 'explanation': 'The candidate does not mention proficiency in Advanced Spanish, which is a mandatory skill for the job. This is a major drawback for their suitability.'}, 'domain': {'feature': 'iGaming and Gambling Experience', 'score': 0, 'explanation': 'The candidate lacks specific experience in iGaming and Gambling, crucial for the position. This absence severely impacts their candidacy.'}, 'location': {'feature': 'Candidate Location', 'score': 0, 'explanation': 'The candidate is located in San Francisco, CA, which is outside the preferred locations listed in the job specification. This significantly decreases their suitability.'}, 'overall_summary': {'score': 0, 'summary': 'The candidate has extensive experience and a strong background in product management, but lacks required language skills (Advanced Spanish) and relevant domain expertise in iGaming and Gambling, while also being outside the preferred location. As such, they are not suited for this position.'}}"
2,292,"{'experience': {'feature': 'Experience', 'score': 5, 'explanation': 'The candidate has approximately 2 years of relevant experience as a Product Owner, which is less than the 3 years required by the job specification. However, they have diverse roles in marketing and project management that may contribute positively.'}, 'skills': {'feature': 'Skills', 'score': 0, 'explanation': 'The candidate does not meet the mandatory skills requirement of advanced Spanish, which is critical for the position. They also lack direct experience in product development relevant to the gaming or gambling domain.'}, 'domain': {'feature': 'Domain Expertise', 'score': 0, 'explanation': 'The candidate has no indicated experience in iGaming or Gambling, which is a must-have for the role.'}, 'location': {'feature': 'Location', 'score': 5, 'explanation': 'The candidate is located in Istanbul, Türkiye, which is not on the preferred location list and negatively impacts the evaluation, although it is still within a region of interest.'}, 'overall_summary': {'score': 10, 'summary': 'Overall, the candidate lacks critical skills and domain expertise for the position, and their location is non-preferred. They have some relevant experience, but it is insufficient compared to the requirements.'}}"
3,293,"{'experience': {'feature': 'Work Experience', 'score': 10, 'explanation': 'The candidate has over 5 years of experience in product management, which exceeds the required 3 years. Overall experience is relevant to product management, but the candidate is not overqualified for the role.'}, 'skills

In [6]:
#now flatten the content column {'experience': {'feature': 'Experience in product management', 'score': 3, 'explanation': 'The candidate has 3 years of experience, transitioning from a Project Manager to a Product Manager, which closely aligns with the required experience level for the role.'}, 'skills': {'feature': 'Relevant skills for product management', 'score': 3, 'explanation': 'The candidate possesses product development and management skills but lacks proficiency in advanced Spanish and knowledge of Latin American markets, which are essential requirements.'}, 'domain': {'feature': 'Experience in the iGaming or Gambling industry', 'score': 0, 'explanation': 'The candidate does not mention any experience in the iGaming or Gambling domains, which is a necessary criterion for this role.'}, 'location': {'feature': 'Willingness to relocate from Armenia', 'score': 0, 'explanation': 'The candidate is currently based in Armenia and does not meet the citizenship requirement as they are Armenian.'}, 'overall_summary': {'score': 1, 'summary': 'The candidate demonstrates relevant experience and product management skills, but fails to meet critical requirements including domain experience in iGaming/Gambling, advanced Spanish proficiency, and appropriate citizenship for the role.'}}

# with id

df_results = pd.json_normalize(df_results["content"])
df_results["custom_id"] = df_results.index

df_results

,experience.feature,experience.score,experience.explanation,skills.feature,skills.score,skills.explanation,domain.feature,domain.score,domain.explanation,location.feature,location.score,location.explanation,overall_summary.score,overall_summary.summary,custom_id
0,Experience,5,"The candidate has around 6 years of experience in product management and business development, which exceeds the required 3 years. However, this could indicate overqualification.",Mandatory skills,0,"The candidate does not meet the mandatory skills requirements of 'Product development', 'Advanced Spanish', or 'Native Russian'. This strongly impacts the evaluation.","Domain Expertise in iGaming, Gambling",0,"The candidate has no experience or expertise in the iGaming or Gambling domains, which are critical for this role.",Location,10,"The candidate is located in Mexico, which is one of the preferred locations for the job.",5,"Overall, the candidate has significant experience and is located in a preferred region, but lacks the mandatory skills and domain expertise required for the role.",0
1,Years of Experience,5,"The candidate has 12 years of experience which is significantly above the required 3 years. While this is generally positive, the overqualification could imply a mismatch with expectations.",Mandatory Skills,0,"The candidate does not mention proficiency in Advanced Spanish, which is a mandatory skill for the job. This is a major drawback for their suitability.",iGaming and Gambling Experience,0,"The candidate lacks specific experience in iGaming and Gambling, crucial for the position. This absence severely impacts their candidacy.",Candidate Location,0,"The candidate is located in San Francisco, CA, which is outside the preferred locations listed in the job specification. This significantly decreases their suitability.",0,"The candidate has extensive experience and a strong background in product management, but lacks required language skills (Advanced Spanish) and relevant domain expertise in iGaming and Gambling, while also being outside the preferred location. As such, they are not suited for this position.",1
2,Experience,5,"The candidate has approximately 2 years of relevant experience as a Product Owner, which is less than the 3 years required by the job specification. However, they have diverse roles in marketing and project management that may contribute positively.",Skills,0,"The candidate does not meet the mandatory skills requirement of advanced Spanish, which is critical for the position. They also lack direct experience in product development relevant to the gaming or gambling domain.",Domain Expertise,0,"The candidate has no indicated experience in iGaming or Gambling, which is a must-have for the role.",Location,5,"The candidate is located in Istanbul, Türkiye, which is not on the preferred location list and negatively impacts the evaluation, although it is still within a region of interest.",10,"Overall, the candidate lacks critical skills and domain expertise for the position, and their location is non-preferred. They have some relevant experience, but it is insufficient compared to the requirements.",2
3,Work Experience,10,"The candidate has over 5 years of experience in product management, which exceeds the required 3 years. Overall experience is relevant to product management, but the candidate is not overqualified for the role.",Mandatory Skills,0,"The candidate does not possess advanced Spanish, which is a mandatory skill for the position. This significantly impacts their suitability for the role.",Domain Expertise in iGaming and Gambling,0,"The candidate does not have experience in iGaming or gambling, which are essential domains for the role. This is a critical gap.",Candidate Location,0,"The candidate is located in Thailand, which is not among the preferred locations for the job. This negatively impacts their suitability.",0,"The candidate has strong relevant experience and skills but lacks mandatory Spanish proficiency a

In [7]:
# str id
df["id"] = df.index
df["id"] = df["id"].astype(str)
df

/var/folders/0_/14m9hb7n2234bn6kbkgxxvbr0000gn/T/ipykernel_41284/3821069240.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id"] = df.index
/var/folders/0_/14m9hb7n2234bn6kbkgxxvbr0000gn/T/ipykernel_41284/3821069240.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id"] = df["id"].astype(str)


id  \
0      0   
1      1   
2      2   
3      3   
4      4   
..   ...   
361  361   
362  362   
363  363   
364  364   
365  365   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [8]:

df_results["custom_id"] = df_results["custom_id"].astype(str)
#merge with df
df_results = df_results.merge(df, left_on="custom_id", right_on="id", how="left")
df_results = df_results.drop(columns=["custom_id", "id"])
df_results

experience.feature  experience.score  \
0             Experience                 5   
1    Years of Experience                 5   
2             Experience                 5   
3        Work Experience                10   
4             Experience                 0   
..                   ...               ...   
361      Work Experience                 5   
362           Experience                 5   
363  Years of Experience                 5   
364           Experience                 5   
365           Experience                 0   

                                                                                                                                                                                                                                        experience.explanation  \
0                                                                           The candidate has around 6 years of experience in product management and business development, which exceeds the required 3 years. However, this could indicate overqualification.   
1                                                                The candidate has 12 years of experience which is significantly above the required 3 years. While this is generally positive, the overqualification could imply a mismatch with expectations.   
2    The candidate has approximately 2 years of relevant experience as a Product Owner, which is less than the 3 years required by the job specification. However, they have diverse roles in marketing and project management that may contribute positively.   
3                                           The candidate has over 5 years of experience in product management, which exceeds the required 3 years. Overall experience is relevant to product management, but the candidate is not overqualified for the role.   
4                                             The candidate has over 10 years of experience, which is significantly above the required 3 years. However, overqualification in terms of experience is penalized as it does not align with the job requirements.   
..                                                                                                                                                                                                                                                         ...   
361                                                   The candidate has over 10 years of experience while the job requires 3 years. Although positive, their extensive experience may indicate overqualification which could complicate fitting into the team.   
362                                     The candidate has 15+ years of experience, which is significantly over the 3 years required. While this showcases extensive experience, it also suggests potential overqualification, thus the score is adjusted to 5.   
363                                                      The candidate has 8 years of experience, which is more than the required 3 years. While this is generally positive, overqualification in experience for the role may not align with the expectations.   
364            The candidate has over 15 years of experience in the gaming industry, which is significantly more than the required 3 years. However, their extensive experience could be seen as overqualification for the role, leading to a mid-range score.   
365                                                                 The candidate has over 11 years of experience, which is significantly more than the 3 years required. However, this level of overqualification may create concerns about fit for the role.   

                   skills.feature  skills.score  \
0                Mandatory skills             0   
1                Mandatory Skills             0   
2                          Skills             0   
3                Mandatory Skills             0   
4                          Skills             0   
..                            ...           ...   

In [9]:
df_results.to_csv("results.csv", index=False)